# Create Project

## Load Libraries and Functions

In [1]:
# import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Load Data

In [2]:
# imdb urls for datasets
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [22]:
# loading the data
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# viewing the data
display(basics.head(), akas.head(), ratings.head())


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


,tconst,averageRating,numVotes
0,tt0000001,5.7,1980
1,tt0000002,5.8,265
2,tt0000003,6.5,1833
3,tt0000004,5.6,179
4,tt0000005,6.2,2622


## Cleaning the Data

In [4]:
# checking changes
display(basics.head(3), akas.head(3))

# ratings did not appear to have any NaN values
ratings.isna().sum()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0


tconst           0
averageRating    0
numVotes         0
dtype: int64

### Basics
* [x] Eliminate movies that are null for runtimeMinutes
* [x] Eliminate movies that are null for genre
* [x] keep only titleType==Movie
* [x] keep startYear 2000-2022
* [x] Eliminate movies that include "Documentary" in genre (see tip below)
* [x] Keep only US movies

In [ ]:
#load basics
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)


In [5]:
# look at info
basics.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9926879 entries, 0 to 9926878
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 681.6+ MB


In [12]:
# replace null values
basics.replace({'\\N':np.nan}, inplace = True)

In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9926879 entries, 0 to 9926878
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 681.6+ MB


In [16]:
# 'startYear' is a string column, changing it to an float (to account for NaNs)
basics['startYear'] = basics['startYear'].astype(float)

# confirming
basics.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult            object
startYear         float64
endYear            object
runtimeMinutes     object
genres             object
dtype: object

In [17]:
#  drop minutes and genres 
basics= basics.dropna(subset= ['runtimeMinutes', 'genres'], inplace=True)

In [8]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9926879 entries, 0 to 9926878
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 681.6+ MB


In [18]:
#drop movie
basics = basics[(basics['titleType'] == 'movie')]

TypeError: 'NoneType' object is not subscriptable

In [20]:
# keep only 'Movie'
basics = basics.loc[ basics['titleType']=='movie']

AttributeError: 'NoneType' object has no attribute 'loc'

In [21]:
basics.info()

AttributeError: 'NoneType' object has no attribute 'info'